<a href="https://colab.research.google.com/github/Edu963/ichimoku/blob/main/new_ichi_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt

ModuleNotFoundError: No module named 'vectorbt'

In [ ]:
# Download historical stock price data
df = yf.download("COIN", start="2024-04-01", end="2024-12-31", interval='15m')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Define the Ichimoku Cloud function
def ichimoku_cloud(df, n_delay=26, k_delay=9, lag_span=52):
    # Calculate Tenkan-Sen
    tenkan_sen_high = df['High'].rolling(window=n_delay).max()
    tenkan_sen_low = df['Low'].rolling(window=n_delay).min()
    tenkan_sen = (tenkan_sen_high + tenkan_sen_low) / 2

    # Calculate Kijun-Sen
    kijun_sen_high = df['High'].rolling(window=k_delay).max()
    kijun_sen_low = df['Low'].rolling(window=k_delay).min()
    kijun_sen = (kijun_sen_high + kijun_sen_low) / 2

    # Calculate Senkou Span A
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(k_delay)

    # Calculate Senkou Span B
    lagging_span_high = df['High'].rolling(window=lag_span).max().shift(lag_span)
    lagging_span_low = df['Low'].rolling(window=lag_span).min().shift(lag_span)
    senkou_span_b = ((lagging_span_high + lagging_span_low) / 2).shift(k_delay)

    # Calculate Chikou Span
    chikou_span = df['Close'].shift(-k_delay)

    # Include the 'Close' column in the output
    return pd.DataFrame({
        'Tenkan-Sen': tenkan_sen,
        'Kijun-Sen': kijun_sen,
        'Senkou Span A': senkou_span_a,
        'Senkou Span B': senkou_span_b,
        'Chikou Span': chikou_span,
        'Close': df['Close'],  # Ensure 'Close' column is included
        'Open': df['Open'],    # Include 'Open' column for potential use
        'High': df['High'],
        'Low': df['Low'],
    }, index=df.index)


In [ ]:
# Calculate Ichimoku Cloud
ichimoku = ichimoku_cloud(df)

# Print the Ichimoku Cloud data
print(ichimoku)

                     Tenkan-Sen   Kijun-Sen  Senkou Span A  Senkou Span B  \
Datetime                                                                    
2024-04-01 09:30:00         NaN         NaN            NaN            NaN   
2024-04-01 09:45:00         NaN         NaN            NaN            NaN   
2024-04-01 10:00:00         NaN         NaN            NaN            NaN   
2024-04-01 10:15:00         NaN         NaN            NaN            NaN   
2024-04-01 10:30:00         NaN         NaN            NaN            NaN   
...                         ...         ...            ...            ...   
2024-05-01 12:00:00  203.900002  202.515350     204.044971     227.099998   
2024-05-01 12:15:00  203.034996  202.945351     203.834999     227.740005   
2024-05-01 12:30:00  202.489998  202.945351     203.785000     227.740005   
2024-05-01 12:45:00  202.279999  202.945351     203.785000     227.740005   
2024-05-01 13:00:00  202.254997  202.945351     203.745724     227.840004   

In [ ]:
def ichimoku_entry_signals(ichimoku):
    # Condition 1 for long entry: Check if the candle opens and closes above the cloud
    above_cloud_long = (ichimoku['Close'] > ichimoku['Senkou Span A']) & (ichimoku['Close'] > ichimoku['Senkou Span B'])

    # Condition 2 for long entry: Ensure Tenkan-Sen is above Kijun-Sen
    tenkan_above_kijun_long = ichimoku['Tenkan-Sen'] > ichimoku['Kijun-Sen']

    # Condition 3 for long entry: Check if OHLC values are all above Tenkan-Sen and Kijun-Sen
    ohlc_above_sen_long = (ichimoku['Open'] > ichimoku['Tenkan-Sen']) & \
                     (ichimoku['High'] > ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Low'] > ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Close'] > ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Open'] > ichimoku['Kijun-Sen']) & \
                     (ichimoku['High'] > ichimoku['Kijun-Sen']) & \
                     (ichimoku['Low'] > ichimoku['Kijun-Sen']) & \
                     (ichimoku['Close'] > ichimoku['Kijun-Sen'])

    # Condition 4 for long entry: Check if future cloud is positive (long)
    future_cloud_positive_long = ichimoku['Senkou Span A'] > ichimoku['Senkou Span B']

    # Combine all conditions for long entry
    entry_signal_long = above_cloud_long & tenkan_above_kijun_long & ohlc_above_sen_long & future_cloud_positive_long

    # Condition 1 for short entry: Check if the candle opens and closes below the cloud
    below_cloud_short = (ichimoku['Close'] < ichimoku['Senkou Span A']) & (ichimoku['Close'] < ichimoku['Senkou Span B'])

    # Condition 2 for short entry: Ensure Tenkan-Sen is below Kijun-Sen
    tenkan_below_kijun_short = ichimoku['Tenkan-Sen'] < ichimoku['Kijun-Sen']

    # Condition 3 for short entry: Check if OHLC values are all below Tenkan-Sen and Kijun-Sen
    ohlc_below_sen_short = (ichimoku['Open'] < ichimoku['Tenkan-Sen']) & \
                     (ichimoku['High'] < ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Low'] < ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Close'] < ichimoku['Tenkan-Sen']) & \
                     (ichimoku['Open'] < ichimoku['Kijun-Sen']) & \
                     (ichimoku['High'] < ichimoku['Kijun-Sen']) & \
                     (ichimoku['Low'] < ichimoku['Kijun-Sen']) & \
                     (ichimoku['Close'] < ichimoku['Kijun-Sen'])

    # Condition 4 for short entry: Check if future cloud is negative (short)
    future_cloud_negative_short = ichimoku['Senkou Span A'] < ichimoku['Senkou Span B']

    # Combine all conditions for short entry
    entry_signal_short = below_cloud_short & tenkan_below_kijun_short & ohlc_below_sen_short & future_cloud_negative_short

    # Combine long and short entry signals
    entry_signals = entry_signal_long.astype(int) - entry_signal_short.astype(int)

    return entry_signals



---

In [ ]:
entry_signals = ichimoku_entry_signal(ichimoku)

In [ ]:
# Initialize Portfolio object with entry signals
portfolio = vbt.Portfolio.from_signals(
    close=ichimoku['Close'],
    entries=entry_signals,
    # Add exits parameter if exit signals are defined
)

In [ ]:
# Calculate trade statistics
trades = portfolio.trades

In [ ]:
# Step 5: Evaluate Results
total_trades = trades.count()
winning_trades = trades.winning.count()
losing_trades = trades.losing.count()
# Calculate other performance metrics...


In [ ]:
# Print trade statistics
print("Trade Statistics:")
print("Total Trades:", total_trades)
print("Winning Trades:", winning_trades)
print("Losing Trades:", losing_trades)

Trade Statistics:
Total Trades: 1
Winning Trades: 0
Losing Trades: 1


In [ ]:
# Visualize performance (optional)
portfolio.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'fa7d151b-5ed6-4d0d-99de-56cde33a287a',
              'x': array([datetime.datetime(2024, 4, 1, 9, 30),
                          datetime.datetime(2024, 4, 1, 9, 45),
                          datetime.datetime(2024, 4, 1, 10, 0), ...,
                          datetime.datetime(2024, 5, 1, 12, 30),
                          datetime.datetime(2024, 5, 1, 12, 45),
                          datetime.datetime(2024, 5, 1, 13, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([262.80599976, 268.58401489, 266.8999939 , ..., 205.03999329,
                          205.65499878, 205.72000122]),
              'yaxis': 'y'},
             {'customdata': array([[0.        , 0.45113123, 0.        ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' .

In [ ]:
portfolio.stats()

/home/edu/Documents/trading/Ichimoku/ichi/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/home/edu/Documents/trading/Ichimoku/ichi/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/home/edu/Documents/trading/Ichimoku/ichi/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/home/edu/Documents/trading/Ichimoku/ichi/lib/python3.11/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2024-04-01 09:30:00
End                           2024-05-01 13:00:00
Period                                        587
Start Value                                 100.0
End Value                               92.806716
Total Return [%]                        -7.193284
Benchmark Return [%]                   -21.721726
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        15.348433
Max Drawdown Duration                       153.0
Total Trades                                    1
Total Closed Trades                             0
Total Open Trades                               1
Open Trade PnL                          -7.193284
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN
